In [1]:
import httplib2
import oauth2
import urllib3
import types
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2, psycopg2.extras
import httplib
import base64
import json # For Microsoft Face API
import urllib as urllib # For Microsoft Face API
import time 
import csv
import datetime 

class UpworkDataFormatter:
    
    def __init__(self):
        # Set the present date for collecting the data
        self.current_date = "12/31/2017"
        self.current_date = datetime.datetime.strptime(self.current_date, "%m/%d/%Y")
        
        # File name
        self.actual_rates_file_name = "./csv_files/upwork_actual_rates_2017_12_12.csv"
        self.skilltest_tenure_file_name = "./csv_files/upwork_skilltest_tenure_2017_12_12.csv"
        self.hr_tenure_file_name = "./csv_files/upwork_hr_tenure_2017_12_12.csv"
        self.fp_tenure_file_name = "./csv_files/upwork_fp_tenure_2017_12_12.csv"
        
        # Connect to the database
        self.conn = psycopg2.connect("dbname=eureka01")
        self.cur = self.conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
        psycopg2.extensions.register_adapter(dict, psycopg2.extras.Json)
        self.cur.execute("SELECT detailed_info FROM upwork_unitedstates_allskills_2017_12_12;")
        
        # Save tenure data to file
        self.save_upwork_tenure_to_csv('fp_tenure', self.fp_tenure_file_name)
    
    def save_upwork_tenure_to_csv(self, field, filename):
        
        with open(filename, 'w') as csvfile:
            user_count = 1
            fieldnames = ['user_count', field]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

            for user in self.cur:
                try: 
                    value = self.calculate_assignment_tenure(user, field[:2])
                    writer.writerow({'user_count': user_count, field: value})

                except KeyboardInterrupt:
                    print "We stoppped"
                    break

                except Exception as error:
                    print error
                    writer.writerow({'user_count': user_count, field: "None"})
                
                print "Finished user {0} for {1}".format(user_count, field)
                user_count += 1
                
    # Calculate the earliest skill test date on Upwork, returns Upwork tenure in days, months and years as list
    def calculate_skilltest_tenure(self, user):
        
        try:
            skilltest_list = user[0]["tsexams"]["tsexam"]
        except:
            return "None"
        
        all_dates = []
        earliest_date = ""
        
        if (type(skilltest_list) is list): 
            for test in skilltest_list:
                date = test['ts_when']
                date_datetime = datetime.datetime.strptime(date, "%m/%d/%Y")
                all_dates.append(date_datetime)
            earliest_date = min(all_dates)
        
        elif (type(skilltest_list) is dict):
            date = skilltest_list['ts_when']
            date_datetime = datetime.datetime.strptime(date, "%m/%d/%Y")
            earliest_date = date_datetime
        
        # Calculate number of months on Upwork
        tenure_days = (self.current_date - earliest_date).days
        tenure_months = tenure_days/30
        tenure_years = tenure_days/365
        return [tenure_days, tenure_months, tenure_years]
    
    # Returns Upwork tenure based on earliest end date for a fixed price or hourly assignment, and average pay for hourly assignments 
    def calculate_assignment_tenure(self, user, hr_or_fp): # Set last attribute to hr or fp 
        try: 
            assignments_list = user[0]["assignments"][hr_or_fp]["job"]
        except:
            return "None"
        
        all_end_dates = []
        earliest_date = ""
        
        if (type(assignments_list) is list): # If more than one assignment reported
            for assignment in assignments_list:
                end_date = assignment["as_to_full"]
                if (end_date == "Present"):
                    end_date = self.current_date
                    end_datetime = end_date
                else:
                    end_datetime = datetime.datetime.strptime(end_date, "%m/%d/%Y")
                
                all_end_dates.append(end_datetime)
            earliest_date = min(all_end_dates)

        elif (type(assignments_list) is dict): # If only one assignment reported
            end_date = assignments_list["as_to_full"]
            if (end_date == "Present"):
                end_date = self.current_date
                end_datetime = end_date
            else:
                end_datetime = datetime.datetime.strptime(end_date, "%m/%d/%Y")
            earliest_date = end_datetime
        
        # Calculate tenure from earliest work date
        tenure_days = (self.current_date - earliest_date).days
        tenure_months = tenure_days/30
        tenure_years = tenure_days/365
        
        # Calculate average pay for hourly assignments
        if hr_or_fp == "hr":
            all_rates = []
            mean_rate = 0
            if (type(assignments_list) is list): # If more than one assignment reported
                for assignment in assignments_list:
                    rate = assignment["as_rate"]
                    rate = float(rate[1:])
                    all_rates.append(rate)
                
                mean_rate = round(np.array(all_rates).mean(),2)
                median_rate = round(np.median(np.array(all_rates)), 2)
                
            elif (type(assignments_list) is dict):
                rate = assignments_list["as_rate"]
                rate = float(rate[1:])
                rate = round(rate, 2)
                mean_rate = rate
                median_rate = rate
                all_rates.append(mean_rate)
        elif hr_or_fp == "fp":
            mean_rate = "None"
            all_rates = "None"
            median_rate = "None"
        
        return [tenure_days, tenure_months, tenure_years, all_rates, mean_rate, median_rate]
        

myObject = UpworkDataFormatter()

Finished user 1 for fp_tenure
Finished user 2 for fp_tenure
Finished user 3 for fp_tenure
Finished user 4 for fp_tenure
Finished user 5 for fp_tenure
Finished user 6 for fp_tenure
Finished user 7 for fp_tenure
Finished user 8 for fp_tenure
Finished user 9 for fp_tenure
Finished user 10 for fp_tenure
Finished user 11 for fp_tenure
Finished user 12 for fp_tenure
Finished user 13 for fp_tenure
Finished user 14 for fp_tenure
Finished user 15 for fp_tenure
Finished user 16 for fp_tenure
Finished user 17 for fp_tenure
Finished user 18 for fp_tenure
Finished user 19 for fp_tenure
Finished user 20 for fp_tenure
Finished user 21 for fp_tenure
Finished user 22 for fp_tenure
Finished user 23 for fp_tenure
Finished user 24 for fp_tenure
Finished user 25 for fp_tenure
Finished user 26 for fp_tenure
Finished user 27 for fp_tenure
Finished user 28 for fp_tenure
Finished user 29 for fp_tenure
Finished user 30 for fp_tenure
Finished user 31 for fp_tenure
Finished user 32 for fp_tenure
Finished user 33 

Finished user 357 for fp_tenure
Finished user 358 for fp_tenure
Finished user 359 for fp_tenure
Finished user 360 for fp_tenure
Finished user 361 for fp_tenure
Finished user 362 for fp_tenure
Finished user 363 for fp_tenure
Finished user 364 for fp_tenure
Finished user 365 for fp_tenure
Finished user 366 for fp_tenure
Finished user 367 for fp_tenure
Finished user 368 for fp_tenure
Finished user 369 for fp_tenure
Finished user 370 for fp_tenure
Finished user 371 for fp_tenure
Finished user 372 for fp_tenure
Finished user 373 for fp_tenure
Finished user 374 for fp_tenure
Finished user 375 for fp_tenure
Finished user 376 for fp_tenure
Finished user 377 for fp_tenure
Finished user 378 for fp_tenure
Finished user 379 for fp_tenure
Finished user 380 for fp_tenure
Finished user 381 for fp_tenure
Finished user 382 for fp_tenure
Finished user 383 for fp_tenure
Finished user 384 for fp_tenure
Finished user 385 for fp_tenure
Finished user 386 for fp_tenure
Finished user 387 for fp_tenure
Finished

Finished user 702 for fp_tenure
Finished user 703 for fp_tenure
Finished user 704 for fp_tenure
Finished user 705 for fp_tenure
Finished user 706 for fp_tenure
Finished user 707 for fp_tenure
Finished user 708 for fp_tenure
Finished user 709 for fp_tenure
Finished user 710 for fp_tenure
Finished user 711 for fp_tenure
Finished user 712 for fp_tenure
Finished user 713 for fp_tenure
Finished user 714 for fp_tenure
Finished user 715 for fp_tenure
Finished user 716 for fp_tenure
Finished user 717 for fp_tenure
Finished user 718 for fp_tenure
Finished user 719 for fp_tenure
Finished user 720 for fp_tenure
Finished user 721 for fp_tenure
Finished user 722 for fp_tenure
Finished user 723 for fp_tenure
Finished user 724 for fp_tenure
Finished user 725 for fp_tenure
Finished user 726 for fp_tenure
Finished user 727 for fp_tenure
Finished user 728 for fp_tenure
Finished user 729 for fp_tenure
Finished user 730 for fp_tenure
Finished user 731 for fp_tenure
Finished user 732 for fp_tenure
Finished

Finished user 994 for fp_tenure
Finished user 995 for fp_tenure
Finished user 996 for fp_tenure
Finished user 997 for fp_tenure
Finished user 998 for fp_tenure
Finished user 999 for fp_tenure
Finished user 1000 for fp_tenure
Finished user 1001 for fp_tenure
Finished user 1002 for fp_tenure
Finished user 1003 for fp_tenure
Finished user 1004 for fp_tenure
Finished user 1005 for fp_tenure
Finished user 1006 for fp_tenure
Finished user 1007 for fp_tenure
Finished user 1008 for fp_tenure
Finished user 1009 for fp_tenure
Finished user 1010 for fp_tenure
Finished user 1011 for fp_tenure
Finished user 1012 for fp_tenure
Finished user 1013 for fp_tenure
Finished user 1014 for fp_tenure
Finished user 1015 for fp_tenure
Finished user 1016 for fp_tenure
Finished user 1017 for fp_tenure
Finished user 1018 for fp_tenure
Finished user 1019 for fp_tenure
Finished user 1020 for fp_tenure
Finished user 1021 for fp_tenure
Finished user 1022 for fp_tenure
Finished user 1023 for fp_tenure
Finished user 10

Finished user 1277 for fp_tenure
Finished user 1278 for fp_tenure
Finished user 1279 for fp_tenure
Finished user 1280 for fp_tenure
Finished user 1281 for fp_tenure
Finished user 1282 for fp_tenure
Finished user 1283 for fp_tenure
Finished user 1284 for fp_tenure
Finished user 1285 for fp_tenure
Finished user 1286 for fp_tenure
Finished user 1287 for fp_tenure
Finished user 1288 for fp_tenure
Finished user 1289 for fp_tenure
Finished user 1290 for fp_tenure
Finished user 1291 for fp_tenure
Finished user 1292 for fp_tenure
Finished user 1293 for fp_tenure
Finished user 1294 for fp_tenure
Finished user 1295 for fp_tenure
Finished user 1296 for fp_tenure
Finished user 1297 for fp_tenure
Finished user 1298 for fp_tenure
Finished user 1299 for fp_tenure
Finished user 1300 for fp_tenure
Finished user 1301 for fp_tenure
Finished user 1302 for fp_tenure
Finished user 1303 for fp_tenure
Finished user 1304 for fp_tenure
Finished user 1305 for fp_tenure
Finished user 1306 for fp_tenure
Finished u

...

Finished user 50003 for fp_tenure
Finished user 50004 for fp_tenure
Finished user 50005 for fp_tenure
Finished user 50006 for fp_tenure
Finished user 50007 for fp_tenure
Finished user 50008 for fp_tenure
Finished user 50009 for fp_tenure
Finished user 50010 for fp_tenure
Finished user 50011 for fp_tenure
Finished user 50012 for fp_tenure
Finished user 50013 for fp_tenure
Finished user 50014 for fp_tenure
Finished user 50015 for fp_tenure
Finished user 50016 for fp_tenure
Finished user 50017 for fp_tenure
Finished user 50018 for fp_tenure
Finished user 50019 for fp_tenure
Finished user 50020 for fp_tenure
Finished user 50021 for fp_tenure
Finished user 50022 for fp_tenure
Finished user 50023 for fp_tenure
Finished user 50024 for fp_tenure
Finished user 50025 for fp_tenure
Finished user 50026 for fp_tenure
Finished user 50027 for fp_tenure
Finished user 50028 for fp_tenure
Finished user 50029 for fp_tenure
Finished user 50030 for fp_tenure
Finished user 50031 for fp_tenure
Finished user 

Finished user 51136 for fp_tenure
Finished user 51137 for fp_tenure
Finished user 51138 for fp_tenure
Finished user 51139 for fp_tenure
Finished user 51140 for fp_tenure
Finished user 51141 for fp_tenure
Finished user 51142 for fp_tenure
Finished user 51143 for fp_tenure
Finished user 51144 for fp_tenure
Finished user 51145 for fp_tenure
Finished user 51146 for fp_tenure
Finished user 51147 for fp_tenure
Finished user 51148 for fp_tenure
Finished user 51149 for fp_tenure
Finished user 51150 for fp_tenure
Finished user 51151 for fp_tenure
Finished user 51152 for fp_tenure
Finished user 51153 for fp_tenure
Finished user 51154 for fp_tenure
Finished user 51155 for fp_tenure
Finished user 51156 for fp_tenure
Finished user 51157 for fp_tenure
Finished user 51158 for fp_tenure
Finished user 51159 for fp_tenure
Finished user 51160 for fp_tenure
Finished user 51161 for fp_tenure
Finished user 51162 for fp_tenure
Finished user 51163 for fp_tenure
Finished user 51164 for fp_tenure
Finished user 

Finished user 52494 for fp_tenure
Finished user 52495 for fp_tenure
Finished user 52496 for fp_tenure
Finished user 52497 for fp_tenure
Finished user 52498 for fp_tenure
Finished user 52499 for fp_tenure
Finished user 52500 for fp_tenure
Finished user 52501 for fp_tenure
Finished user 52502 for fp_tenure
Finished user 52503 for fp_tenure
Finished user 52504 for fp_tenure
Finished user 52505 for fp_tenure
Finished user 52506 for fp_tenure
Finished user 52507 for fp_tenure
Finished user 52508 for fp_tenure
Finished user 52509 for fp_tenure
Finished user 52510 for fp_tenure
Finished user 52511 for fp_tenure
Finished user 52512 for fp_tenure
Finished user 52513 for fp_tenure
Finished user 52514 for fp_tenure
Finished user 52515 for fp_tenure
Finished user 52516 for fp_tenure
Finished user 52517 for fp_tenure
Finished user 52518 for fp_tenure
Finished user 52519 for fp_tenure
Finished user 52520 for fp_tenure
Finished user 52521 for fp_tenure
Finished user 52522 for fp_tenure
Finished user 

Finished user 53642 for fp_tenure
Finished user 53643 for fp_tenure
Finished user 53644 for fp_tenure
Finished user 53645 for fp_tenure
Finished user 53646 for fp_tenure
Finished user 53647 for fp_tenure
Finished user 53648 for fp_tenure
Finished user 53649 for fp_tenure
Finished user 53650 for fp_tenure
Finished user 53651 for fp_tenure
Finished user 53652 for fp_tenure
Finished user 53653 for fp_tenure
Finished user 53654 for fp_tenure
Finished user 53655 for fp_tenure
Finished user 53656 for fp_tenure
Finished user 53657 for fp_tenure
Finished user 53658 for fp_tenure
Finished user 53659 for fp_tenure
Finished user 53660 for fp_tenure
Finished user 53661 for fp_tenure
Finished user 53662 for fp_tenure
Finished user 53663 for fp_tenure
Finished user 53664 for fp_tenure
Finished user 53665 for fp_tenure
Finished user 53666 for fp_tenure
Finished user 53667 for fp_tenure
Finished user 53668 for fp_tenure
Finished user 53669 for fp_tenure
Finished user 53670 for fp_tenure
Finished user 

Finished user 54979 for fp_tenure
Finished user 54980 for fp_tenure
Finished user 54981 for fp_tenure
Finished user 54982 for fp_tenure
Finished user 54983 for fp_tenure
Finished user 54984 for fp_tenure
Finished user 54985 for fp_tenure
Finished user 54986 for fp_tenure
Finished user 54987 for fp_tenure
Finished user 54988 for fp_tenure
Finished user 54989 for fp_tenure
Finished user 54990 for fp_tenure
Finished user 54991 for fp_tenure
Finished user 54992 for fp_tenure
Finished user 54993 for fp_tenure
Finished user 54994 for fp_tenure
Finished user 54995 for fp_tenure
Finished user 54996 for fp_tenure
Finished user 54997 for fp_tenure
Finished user 54998 for fp_tenure
Finished user 54999 for fp_tenure
Finished user 55000 for fp_tenure
Finished user 55001 for fp_tenure
Finished user 55002 for fp_tenure
Finished user 55003 for fp_tenure
Finished user 55004 for fp_tenure
Finished user 55005 for fp_tenure
Finished user 55006 for fp_tenure
Finished user 55007 for fp_tenure
Finished user 